In [5]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [6]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [7]:
subject = 'Mato Grosso Do Sul - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [8]:
data = pd.read_csv('2003_01_model_input_MS.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Ano,Mato Grosso do Sul - IDH,Mato Grosso do Sul - Valor Cimento Portland,Mato Grosso Do Sul - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),...,Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado,Mato Grosso do Sul - Desemprego,Mato Grosso Do Sul - Consumo de Cimento (t)
0,2003,0.741626,3.399873,640.534672,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,...,23.562895,15.656458,64.514034,3.671686,4.562945e+07,2.151917e+06,16.203179,3.957545e+07,8.248824,417.592000
1,2004,0.742636,3.085211,650.203551,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,...,23.507107,15.723355,65.090391,3.618734,4.632078e+07,2.171425e+06,16.238609,3.976863e+07,8.178155,466.165000
2,2005,0.743635,2.624413,663.501150,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,...,23.449531,15.811007,65.660841,3.545208,4.700352e+07,2.187532e+06,16.294466,3.998653e+07,8.157501,479.481000
3,2006,0.744622,2.675575,712.049517,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,...,23.435028,15.674457,65.475431,3.479776,4.770379e+07,2.210487e+06,16.382367,4.026684e+07,8.070117,515.485000
4,2007,0.745596,3.084080,783.793548,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,...,23.460325,15.674369,65.471227,3.455222,4.835689e+07,2.229287e+06,16.460417,4.051516e+07,8.040811,632.929000
5,2008,0.746558,3.433682,831.774473,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,...,23.505311,16.064850,66.686091,3.455270,4.895245e+07,2.250403e+06,16.533879,4.072450e+07,8.011861,814.771000
6,2009,0.747506,3.651735,863.877907,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,...,23.414338,15.953541,66.658540,3.422201,4.943099e+07,2.269266e+06,16.565699,4.085410e+07,8.040524,693.327000
7,2010,0.748442,3.846003,916.677483,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,...,23.460429,15.531732,65.002039,3.316282,5.089988e+07,2.290154e+06,16.701299,4.138190e+07,8.003862,813.149000
8,2011,0.749364,4.020724,958.130317,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,...,23.635226,15.687149,64.896779,3.223002,5.224283e+07,2.294876e+06,16.813003,4.180160e+07,7.993144,923.248000
9,2012,0.750034,4.183487,956.016292,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,...,23.663605,15.955379,65.719662,3.178068,5.345401e+07,2.288752e+06,16.887782,4.213702e+07,6.360445,1008.624000


In [9]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,Mato Grosso do Sul - IDH,Mato Grosso do Sul - Valor Cimento Portland,Mato Grosso Do Sul - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado,Mato Grosso do Sul - Desemprego
0,0.741626,3.399873,640.534672,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,...,1409.576176,23.562895,15.656458,64.514034,3.671686,4.562945e+07,2.151917e+06,16.203179,3.957545e+07,8.248824
1,0.742636,3.085211,650.203551,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,...,1411.547450,23.507107,15.723355,65.090391,3.618734,4.632078e+07,2.171425e+06,16.238609,3.976863e+07,8.178155
2,0.743635,2.624413,663.501150,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,...,1405.006613,23.449531,15.811007,65.660841,3.545208,4.700352e+07,2.187532e+06,16.294466,3.998653e+07,8.157501
3,0.744622,2.675575,712.049517,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,...,1399.165473,23.435028,15.674457,65.475431,3.479776,4.770379e+07,2.210487e+06,16.382367,4.026684e+07,8.070117
4,0.745596,3.084080,783.793548,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,...,1395.609891,23.460325,15.674369,65.471227,3.455222,4.835689e+07,2.229287e+06,16.460417,4.051516e+07,8.040811
5,0.746558,3.433682,831.774473,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,...,1388.122906,23.505311,16.064850,66.686091,3.455270,4.895245e+07,2.250403e+06,16.533879,4.072450e+07,8.011861
6,0.747506,3.651735,863.877907,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,...,1396.303897,23.414338,15.953541,66.658540,3.422201,4.943099e+07,2.269266e+06,16.565699,4.085410e+07,8.040524
7,0.748442,3.846003,916.677483,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,...,1428.326267,23.460429,15.531732,65.002039,3.316282,5.089988e+07,2.290154e+06,16.701299,4.138190e+07,8.003862
8,0.749364,4.020724,958.130317,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,...,1433.680808,23.635226,15.687149,64.896779,3.223002,5.224283e+07,2.294876e+06,16.813003,4.180160e+07,7.993144
9,0.750034,4.183487,956.016292,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,...,1421.572593,23.663605,15.955379,65.719662,3.178068,5.345401e+07,2.288752e+06,16.887782,4.213702e+07,6.360445


In [10]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      466.165000
1      479.481000
2      515.485000
3      632.929000
4      814.771000
5      693.327000
6      813.149000
7      923.248000
8     1008.624000
9     1028.608000
10     995.222143
11     951.041314
12     904.537543
13     870.341000
14     869.680000
15     902.144000
16    1038.045000
17    1092.056000
18    1152.060000
19            NaN
Name: Mato Grosso Do Sul - Consumo de Cimento (t), dtype: float64

In [11]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Mato Grosso do Sul - IDH,Mato Grosso do Sul - Valor Cimento Portland,Mato Grosso Do Sul - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado,Mato Grosso do Sul - Desemprego
0,-1.668612,-0.519867,-1.441613,2.057317,3.335319,2.503268,1.984588,-1.915550,-2.217697,2.670619,...,0.585040,-0.512799,-0.995119,-1.275766,1.536355,-1.582064,-2.203949,-1.714605,-1.673320,1.153654
1,-1.448585,-1.108989,-1.351083,1.041262,1.436308,0.974434,0.428265,-1.603622,-1.796936,1.398420,...,0.623438,-0.734958,-0.826926,-0.731800,1.345276,-1.407665,-1.729420,-1.593603,-1.506631,1.036910
2,-1.231160,-1.971711,-1.226579,-0.382310,0.608392,1.584406,-0.633957,-1.305339,-1.387550,0.057571,...,0.496031,-0.964232,-0.606552,-0.193409,1.079956,-1.235434,-1.337617,-1.402837,-1.318613,1.002789
3,-1.016392,-1.875923,-0.772023,-0.655123,0.362019,0.700171,0.047872,-1.020736,-0.988421,0.196419,...,0.382254,-1.021988,-0.949866,-0.368399,0.843844,-1.058781,-0.779246,-1.102634,-1.076732,0.858432
4,-0.804333,-1.111106,-0.100288,-0.562655,0.214378,-0.043890,-0.213672,-0.749851,-0.597180,0.430422,...,0.312996,-0.921250,-0.950087,-0.372367,0.755239,-0.894024,-0.321957,-0.836074,-0.862463,0.810019
5,-0.595037,-0.456568,0.348954,-0.668618,-0.171237,0.095846,-0.882129,-0.492719,-0.230290,0.038818,...,0.167159,-0.742108,0.031660,0.774223,0.755413,-0.743785,0.191691,-0.585184,-0.681830,0.762193
6,-0.388558,-0.048322,0.649537,-0.307381,-0.482640,-0.506598,-0.409438,-0.249377,0.100246,-0.139605,...,0.326514,-1.104376,-0.248193,0.748220,0.636084,-0.623067,0.650524,-0.476510,-0.569994,0.809545
7,-0.184948,0.315394,1.143896,-0.103220,-0.675189,-0.543480,-0.165137,0.025012,0.397049,0.089437,...,0.950267,-0.920837,-1.308705,-0.815186,0.253870,-0.252515,1.158607,-0.013403,-0.114566,0.748980
8,0.015739,0.642513,1.532016,-0.057995,-0.608258,-0.108258,-0.384906,0.201698,0.638927,0.151742,...,1.054566,-0.224765,-0.917955,-0.914531,-0.082731,0.086265,1.273468,0.368094,0.247582,0.731273
9,0.161537,0.947243,1.512223,0.191223,-0.550547,-0.852408,-0.436543,0.383423,0.817429,0.185184,...,0.818714,-0.111757,-0.243572,-0.137894,-0.244877,0.391805,1.124521,0.623483,0.537011,-1.965926


In [12]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0      466.165000
1      479.481000
2      515.485000
3      632.929000
4      814.771000
5      693.327000
6      813.149000
7      923.248000
8     1008.624000
9     1028.608000
10     995.222143
11     951.041314
12     904.537543
13     870.341000
14     869.680000
15     902.144000
16    1038.045000
Name: Mato Grosso Do Sul - Consumo de Cimento (t), dtype: float64

In [13]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Mato Grosso do Sul - IDH,Mato Grosso do Sul - Valor Cimento Portland,Mato Grosso Do Sul - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado,Mato Grosso do Sul - Desemprego
17,0.566926,2.328712,0.362279,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,...,-0.043668,2.238332,-0.568423,-2.317,-2.866719,0.898913,-0.243065,0.698402,0.787591,-1.313749


In [14]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    1092.056
Name: Mato Grosso Do Sul - Consumo de Cimento (t), dtype: float64

In [15]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    validation_size = len(train_input)//5
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-validation_size:], 
                                       train_target.iloc[-validation_size:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [16]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [18]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[437457431, 839137356, 2583841706, 4076556768, 3381584582, 1944565832, 1878138467, 277775896, 2750684345, 23644611, 947193540, 1270886862, 4179774696, 2150588402, 3434689295, 1939398884, 545150679, 3463701275, 497885838, 1287754047, 3361375980, 2278512449, 1387680426, 2944322802, 2941479338, 1085827055, 691838316, 1409056627, 506079578, 393651237, 570476919, 2683191403, 2335238192, 2706967043, 1357860227, 3419374735, 1086857593, 698189515, 1123164077, 1207628203, 1296652448, 2551786844, 567068418, 1874104987, 3576942135, 2688984188, 1801249314, 934932693, 2200718188, 2748123637, 584496455, 841714436, 2253199195, 1864229592, 1727854994, 2032132024, 3145330579, 3360827510, 1069237731, 4246666891, 1748519685, 4217452891, 1262376883, 1513956250, 242943662, 2732357433, 3945855509, 2956570187, 3101533546, 701100248, 2711665394, 4004916408, 961655320, 2013027273, 331421986, 1415354397, 1366639509, 542625226, 1293620835, 1046778468, 928702922, 3918828929, 840255407, 1711435482, 4164254418, 123

2023-10-04 15:20:21.426683: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-04 15:20:21.456681: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-04 15:20:21.813428: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 5318.86962890625


Step: 15 ___________________________________________
val_loss: 5318.86669921875


Step: 16 ___________________________________________


2023-10-04 15:20:45.741122: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 5319.08837890625


Step: 17 ___________________________________________
val_loss: 1699.8818359375


Step: 18 ___________________________________________
val_loss: 5319.08740234375


Step: 19 ___________________________________________
val_loss: 5318.86669921875


Step: 20 ___________________________________________
val_loss: 4797.998046875


Step: 21 ___________________________________________
val_loss: 2544.4990234375


Step: 22 ___________________________________________
val_loss: 4007.544189453125


Step: 23 ___________________________________________
val_loss: 4449.2939453125


Step: 24 ___________________________________________
val_loss: 2337.279541015625


Step: 25 ___________________________________________
val_loss: 231.5814666748047
winner_seed: 1085827055


Step: 26 ___________________________________________
val_loss: 5318.86669921875


Step: 27 ___________________________________________
val_loss: 5318.86962890625


Step: 28 ______________________________________

...

val_loss: 1322.9678955078125


Step: 59 ___________________________________________
val_loss: 4918.80810546875


Step: 60 ___________________________________________
val_loss: 5319.884765625


Step: 61 ___________________________________________
val_loss: 5318.8681640625


Step: 62 ___________________________________________
val_loss: 1866.9051513671875


Step: 63 ___________________________________________
val_loss: 5318.8671875


Step: 64 ___________________________________________
val_loss: 1761.9190673828125


Step: 65 ___________________________________________
val_loss: 2842.8173828125


Step: 66 ___________________________________________
val_loss: 3941.514404296875


Step: 67 ___________________________________________
val_loss: 2166.016845703125


Step: 68 ___________________________________________
val_loss: 1144.1497802734375


Step: 69 ___________________________________________
val_loss: 793.245849609375


Step: 70 ___________________________________________
val_loss: 5318

2023-10-04 15:39:49.421145: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-04 15:39:49.453153: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 5318.86669921875


Step: 80 ___________________________________________
val_loss: 3466.559814453125


Step: 81 ___________________________________________
val_loss: 3001.954345703125


Step: 82 ___________________________________________
val_loss: 764.2280883789062


Step: 83 ___________________________________________
val_loss: 4733.55322265625


Step: 84 ___________________________________________


2023-10-04 15:41:28.300146: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-04 15:41:28.353713: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 5137.43212890625


Step: 85 ___________________________________________
val_loss: 5958.59228515625


Step: 86 ___________________________________________
val_loss: 5319.21337890625


Step: 87 ___________________________________________
val_loss: 4619.50634765625


Step: 88 ___________________________________________
val_loss: 5318.86669921875


Step: 89 ___________________________________________
val_loss: 3780.4599609375


Step: 90 ___________________________________________
val_loss: 1896.6162109375


Step: 91 ___________________________________________
val_loss: 5318.86669921875


Step: 92 ___________________________________________
val_loss: 5318.87353515625


Step: 93 ___________________________________________
val_loss: 3987.9228515625


Step: 94 ___________________________________________
val_loss: 3502.039794921875


Step: 95 ___________________________________________
val_loss: 4254.51171875


Step: 96 ___________________________________________
val_loss: 29965.51562

In [19]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 467ms/step - loss: 703896.2500 - val_loss: 844370.5000
Epoch 2/10000
1/1 [==============================] - 0s 14ms/step - loss: 686075.9375 - val_loss: 1249533.0000
Epoch 3/10000
1/1 [==============================] - 0s 14ms/step - loss: 693953.7500 - val_loss: 900800.1875
Epoch 4/10000
1/1 [==============================] - 0s 14ms/step - loss: 638251.1250 - val_loss: 542447.0625
Epoch 5/10000
1/1 [==============================] - 0s 14ms/step - loss: 525630.1875 - val_loss: 686603.9375
Epoch 6/10000
1/1 [==============================] - 0s 14ms/step - loss: 494780.1250 - val_loss: 502842.0000
Epoch 7/10000
1/1 [==============================] - 0s 14ms/step - loss: 454778.1562 - val_loss: 907266.5000
Epoch 8/10000
1/1 [==============================] - 0s 14ms/step - loss: 484200.3750 - val_loss: 490827.2500
Epoch 9/10000
1/1 [==============================] - 0s 14ms/step - loss: 421097.7812 - val_loss: 897451.0625
Epoch 10

Epoch 75/10000
1/1 [==============================] - 0s 15ms/step - loss: 142242.6562 - val_loss: 167161.6719
Epoch 76/10000
1/1 [==============================] - 0s 14ms/step - loss: 143350.3438 - val_loss: 165300.8281
Epoch 77/10000
1/1 [==============================] - 0s 14ms/step - loss: 137822.3281 - val_loss: 165376.1719
Epoch 78/10000
1/1 [==============================] - 0s 14ms/step - loss: 130734.9297 - val_loss: 142888.9688
Epoch 79/10000
1/1 [==============================] - 0s 14ms/step - loss: 135058.2188 - val_loss: 178583.3594
Epoch 80/10000
1/1 [==============================] - 0s 14ms/step - loss: 131469.5000 - val_loss: 140453.7812
Epoch 81/10000
1/1 [==============================] - 0s 14ms/step - loss: 126594.8125 - val_loss: 150999.1406
Epoch 82/10000
1/1 [==============================] - 0s 14ms/step - loss: 126336.9297 - val_loss: 137361.0156
Epoch 83/10000
1/1 [==============================] - 0s 15ms/step - loss: 124772.5469 - val_loss: 148260.9375
E

1/1 [==============================] - 0s 14ms/step - loss: 46432.7734 - val_loss: 41648.8320
Epoch 150/10000
1/1 [==============================] - 0s 14ms/step - loss: 42364.9023 - val_loss: 41465.3711
Epoch 151/10000
1/1 [==============================] - 0s 14ms/step - loss: 44177.3516 - val_loss: 37966.8711
Epoch 152/10000
1/1 [==============================] - 0s 14ms/step - loss: 43465.1445 - val_loss: 37637.8672
Epoch 153/10000
1/1 [==============================] - 0s 14ms/step - loss: 41112.5859 - val_loss: 41398.3047
Epoch 154/10000
1/1 [==============================] - 0s 14ms/step - loss: 42333.0000 - val_loss: 35602.4102
Epoch 155/10000
1/1 [==============================] - 0s 14ms/step - loss: 41806.7422 - val_loss: 36701.6992
Epoch 156/10000
1/1 [==============================] - 0s 14ms/step - loss: 37953.3008 - val_loss: 44931.3164
Epoch 157/10000
1/1 [==============================] - 0s 14ms/step - loss: 41521.7148 - val_loss: 33964.9102
Epoch 158/10000
1/1 [=====

1/1 [==============================] - 0s 14ms/step - loss: 26829.9844 - val_loss: 19444.4922
Epoch 224/10000
1/1 [==============================] - 0s 15ms/step - loss: 23183.4121 - val_loss: 15594.1162
Epoch 225/10000
1/1 [==============================] - 0s 14ms/step - loss: 25888.4785 - val_loss: 18546.2988
Epoch 226/10000
1/1 [==============================] - 0s 14ms/step - loss: 20562.9082 - val_loss: 13722.4463
Epoch 227/10000
1/1 [==============================] - 0s 14ms/step - loss: 22640.8223 - val_loss: 18495.1777
Epoch 228/10000
1/1 [==============================] - 0s 14ms/step - loss: 20789.4844 - val_loss: 13892.7109
Epoch 229/10000
1/1 [==============================] - 0s 14ms/step - loss: 24452.9453 - val_loss: 17481.1895
Epoch 230/10000
1/1 [==============================] - 0s 14ms/step - loss: 21784.3691 - val_loss: 14161.1484
Epoch 231/10000
1/1 [==============================] - 0s 14ms/step - loss: 19661.9336 - val_loss: 14023.9971
Epoch 232/10000
1/1 [=====

1/1 [==============================] - 0s 14ms/step - loss: 7099.5576 - val_loss: 7963.3696
Epoch 299/10000
1/1 [==============================] - 0s 14ms/step - loss: 13437.7334 - val_loss: 14226.4336
Epoch 300/10000
1/1 [==============================] - 0s 14ms/step - loss: 14084.0518 - val_loss: 13502.9170
Epoch 301/10000
1/1 [==============================] - 0s 14ms/step - loss: 13137.1641 - val_loss: 14817.6670
Epoch 302/10000
1/1 [==============================] - 0s 14ms/step - loss: 10251.5391 - val_loss: 8055.7026
Epoch 303/10000
1/1 [==============================] - 0s 14ms/step - loss: 17625.3730 - val_loss: 13870.7412
Epoch 304/10000
1/1 [==============================] - 0s 14ms/step - loss: 19531.0469 - val_loss: 13410.0283
Epoch 305/10000
1/1 [==============================] - 0s 14ms/step - loss: 15649.1211 - val_loss: 12447.9424
Epoch 306/10000
1/1 [==============================] - 0s 14ms/step - loss: 16098.3623 - val_loss: 11925.7295
Epoch 307/10000
1/1 [========

...

1/1 [==============================] - 0s 14ms/step - loss: 19838.6973 - val_loss: 5373.3413
Epoch 675/10000
1/1 [==============================] - 0s 14ms/step - loss: 19977.9531 - val_loss: 5331.3921
Epoch 676/10000
1/1 [==============================] - 0s 14ms/step - loss: 19968.3574 - val_loss: 5339.4829
Epoch 677/10000
1/1 [==============================] - 0s 14ms/step - loss: 19966.3281 - val_loss: 5336.5723
Epoch 678/10000
1/1 [==============================] - 0s 15ms/step - loss: 19964.8945 - val_loss: 5336.8750
Epoch 679/10000
1/1 [==============================] - 0s 14ms/step - loss: 19963.5273 - val_loss: 5336.3281
Epoch 680/10000
1/1 [==============================] - 0s 14ms/step - loss: 19996.5020 - val_loss: 5334.4404
Epoch 681/10000
1/1 [==============================] - 0s 15ms/step - loss: 19988.8262 - val_loss: 5335.0044
Epoch 682/10000
1/1 [==============================] - 0s 14ms/step - loss: 20147.9434 - val_loss: 5322.0776
Epoch 683/10000
1/1 [==============

1/1 [==============================] - 0s 15ms/step - loss: 19893.9961 - val_loss: 5327.3691
Epoch 750/10000
1/1 [==============================] - 0s 14ms/step - loss: 19713.4727 - val_loss: 6552.7983
Epoch 751/10000
1/1 [==============================] - 0s 14ms/step - loss: 21348.6953 - val_loss: 5428.9419
Epoch 752/10000
1/1 [==============================] - 0s 14ms/step - loss: 18932.2539 - val_loss: 5347.5757
Epoch 753/10000
1/1 [==============================] - 0s 14ms/step - loss: 18769.5742 - val_loss: 5319.9800
Epoch 754/10000
1/1 [==============================] - 0s 14ms/step - loss: 20071.9160 - val_loss: 5319.7500
Epoch 755/10000
1/1 [==============================] - 0s 14ms/step - loss: 18744.6230 - val_loss: 5327.2891
Epoch 756/10000
1/1 [==============================] - 0s 15ms/step - loss: 20078.5156 - val_loss: 5320.7979
Epoch 757/10000
1/1 [==============================] - 0s 14ms/step - loss: 20066.6973 - val_loss: 5318.8896
Epoch 758/10000
1/1 [==============

1/1 [==============================] - 0s 15ms/step - loss: 14372.1152 - val_loss: 5665.5913
Epoch 825/10000
1/1 [==============================] - 0s 15ms/step - loss: 16389.5664 - val_loss: 5427.9722
Epoch 826/10000
1/1 [==============================] - 0s 14ms/step - loss: 16306.4912 - val_loss: 5482.5684
Epoch 827/10000
1/1 [==============================] - 0s 15ms/step - loss: 15513.2900 - val_loss: 5639.1328
Epoch 828/10000
1/1 [==============================] - 0s 15ms/step - loss: 16239.6436 - val_loss: 5437.3647
Epoch 829/10000
1/1 [==============================] - 0s 15ms/step - loss: 16167.7627 - val_loss: 5485.5195
Epoch 830/10000
1/1 [==============================] - 0s 14ms/step - loss: 15356.9902 - val_loss: 5654.1694
Epoch 831/10000
1/1 [==============================] - 0s 14ms/step - loss: 15283.8271 - val_loss: 5606.3906
Epoch 832/10000
1/1 [==============================] - 0s 15ms/step - loss: 15225.9854 - val_loss: 5623.4478
Epoch 833/10000
1/1 [==============

1/1 [==============================] - 0s 14ms/step - loss: 19930.7734 - val_loss: 5319.3149
Epoch 900/10000
1/1 [==============================] - 0s 14ms/step - loss: 20361.0469 - val_loss: 5335.8501
Epoch 901/10000
1/1 [==============================] - 0s 14ms/step - loss: 19902.5508 - val_loss: 5327.5962
Epoch 902/10000
1/1 [==============================] - 0s 14ms/step - loss: 19874.2793 - val_loss: 5319.8594
Epoch 903/10000
1/1 [==============================] - 0s 14ms/step - loss: 19872.1406 - val_loss: 5321.2480
Epoch 904/10000
1/1 [==============================] - 0s 15ms/step - loss: 20377.7559 - val_loss: 5341.7891
Epoch 905/10000
1/1 [==============================] - 0s 15ms/step - loss: 19909.8906 - val_loss: 5328.4692
Epoch 906/10000
1/1 [==============================] - 0s 14ms/step - loss: 19874.7773 - val_loss: 5319.7056
Epoch 907/10000
1/1 [==============================] - 0s 14ms/step - loss: 20371.9785 - val_loss: 5340.6089
Epoch 908/10000
1/1 [==============

1/1 [==============================] - 0s 14ms/step - loss: 19935.7832 - val_loss: 5332.4692
Epoch 975/10000
1/1 [==============================] - 0s 14ms/step - loss: 19875.1289 - val_loss: 5319.5347
Epoch 976/10000
1/1 [==============================] - 0s 14ms/step - loss: 19870.4121 - val_loss: 5321.5044
Epoch 977/10000
1/1 [==============================] - 0s 15ms/step - loss: 19870.0137 - val_loss: 5320.8530
Epoch 978/10000
1/1 [==============================] - 0s 15ms/step - loss: 19869.9512 - val_loss: 5321.0513
Epoch 979/10000
1/1 [==============================] - 0s 15ms/step - loss: 19869.9102 - val_loss: 5321.0225
Epoch 980/10000
1/1 [==============================] - 0s 15ms/step - loss: 19688.3164 - val_loss: 5378.8433
Epoch 981/10000
1/1 [==============================] - 0s 15ms/step - loss: 20476.9590 - val_loss: 5361.1724
Epoch 982/10000
1/1 [==============================] - 0s 15ms/step - loss: 19933.6289 - val_loss: 5332.5981
Epoch 983/10000
1/1 [==============

In [20]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 57ms/step
[1092.056](test_target) - [[1043.9937]](prediction) = 48.06234765625004


In [21]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [22]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,Mato Grosso do Sul - IDH,Mato Grosso do Sul - Valor Cimento Portland,Mato Grosso Do Sul - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado,Mato Grosso do Sul - Desemprego
0,-1.546922,0.426496,-1.180396,2.184461,2.272299,1.914650,2.360960,-1.590751,-1.593283,2.299346,...,0.460889,1.977871,-0.643856,-1.470281,1.663621,-1.486090,-1.500496,-1.347217,-1.431223,1.847462
1,-1.093952,-0.341244,-1.083070,1.077411,0.707092,0.380286,0.491065,-1.095308,-1.106645,0.891536,...,0.630113,0.731408,-0.232718,-0.667653,1.163491,-1.059395,-1.070923,-1.129572,-1.089335,1.004430
2,-0.646341,-1.465540,-0.949218,-0.473651,0.024704,0.992464,-0.785177,-0.621537,-0.633164,-0.592240,...,0.068614,-0.554973,0.305976,0.126749,0.469042,-0.638006,-0.716239,-0.786442,-0.703703,0.758035
3,-0.204199,-1.340709,-0.460535,-0.770896,-0.178361,0.105030,0.034030,-0.169496,-0.171546,-0.438592,...,-0.432819,-0.879017,-0.533237,-0.131451,-0.148957,-0.205797,-0.210768,-0.246470,-0.207592,-0.284392
4,0.232365,-0.344003,0.261633,-0.670147,-0.300051,-0.641723,-0.280211,0.260758,0.280950,-0.179645,...,-0.738048,-0.313815,-0.533778,-0.137306,-0.380874,0.197306,0.203199,0.232989,0.231883,-0.633996
5,0.663241,0.508987,0.744604,-0.785600,-0.617883,-0.501481,-1.083350,0.669167,0.705281,-0.612992,...,-1.380769,0.691294,1.866050,1.554499,-0.380416,0.564886,0.668184,0.684262,0.602369,-0.979352
6,1.088319,1.041011,1.067754,-0.392011,-0.874548,-1.106104,-0.515420,1.055674,1.087568,-0.810435,...,-0.678471,-1.341271,1.181964,1.516132,-0.692749,0.860243,1.083547,0.879733,0.831749,-0.637417
7,1.507490,1.515004,1.599229,-0.169566,-1.033252,-1.143120,-0.221898,1.491492,1.430840,-0.556978,...,2.070492,-0.311497,-1.410401,-0.790691,-1.693158,1.766854,1.543496,1.712719,1.765852,-1.074770


train_target:


,Mato Grosso Do Sul - Consumo de Cimento (t)
0,466.165
1,479.481
2,515.485
3,632.929
4,814.771
5,693.327
6,813.149
7,923.248


test_input:


...

,Mato Grosso do Sul - IDH,Mato Grosso do Sul - Valor Cimento Portland,Mato Grosso Do Sul - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado,Mato Grosso do Sul - Desemprego
17,0.566926,2.328712,0.362279,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,...,-0.043668,2.238332,-0.568423,-2.317,-2.866719,0.898913,-0.243065,0.698402,0.787591,-1.313749


test_target:


,Mato Grosso Do Sul - Consumo de Cimento (t)
17,1092.056


1/1 [==============================] - 0s 34ms/step
Error: 48.06234765625004




[823.0115966796875,
 867.88037109375,
 938.5494384765625,
 1008.368896484375,
 989.390380859375,
 867.8365478515625,
 887.4078369140625,
 877.54638671875,
 881.5565185546875,
 1043.99365234375]

In [23]:
display(targets)
display(predictions)

[1008.624,
 1028.608,
 995.2221428571428,
 951.0413142857142,
 904.5375428571429,
 870.341,
 869.68,
 902.144,
 1038.045,
 1092.056]

[823.0115966796875,
 867.88037109375,
 938.5494384765625,
 1008.368896484375,
 989.390380859375,
 867.8365478515625,
 887.4078369140625,
 877.54638671875,
 881.5565185546875,
 1043.99365234375]

In [24]:
mae = mean_absolute_error(predictions, targets)
mae

79.45738882533485

In [25]:
porcentage = mae/np.mean(targets)
porcentage

0.0822514798199671